# Acesso ao banco SQL Server

In [ ]:
from __future__ import division
import pymssql
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
from InfosVariaveis import caracteristicas
from pylab import plot, ylim, xlim, show, xlabel, ylabel, grid
from numpy import linspace, loadtxt, ones, convolve

In [ ]:
import pymssql
ini = time.time()
conn = pymssql.connect(host='NTB032\\NTB032',user='sa',password='bdrc2018',database='ISOTROL',as_dict=False,charset='UTF-8')

In [1]:
### MEDIA MOVEL ###
def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
gamesaP=[18,87,208,383,620,938,1348,1743,1937,1988,1998,2000,2000,2000,2000,2000,2000,2000,2000,1906,1681,1455,1230]
gamesaW=[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]

### Query para importar nomes das tabelas e excluir as não necessárias

In [ ]:
cur1 = conn.cursor()
cur1.execute("SELECT name FROM sys.tables;")
tables = cur1.fetchall()
del(tables[40], tables[40], tables[40], tables[-1])
cur1.close()

### Importar todas as variáveis e salvar no arquivo 

In [ ]:
for k in tables:
    if k[0] not in caracteristicas.keys():
        print(str(k[0]))
        cur2 = conn.cursor()
        cur2.execute("SELECT * "
                    "FROM ["+str(k[0])+"]"
                    "ORDER BY TimeStampConvert;")
        value = []
        timestamp = []
        for row in cur2:
            value.append(row[1])
            timestamp.append(row[3])
        cur2.close()

        value = np.array(value)
        timestamp = np.array(timestamp)
        name = row[0]+"_A01"
        
        x = timestamp
        y = value

        plt.scatter(x,y,c="k", alpha=0.5)
        y_av = movingaverage(y, 1000)
        plt.plot(x, y_av,"r")
        plt.xlabel("Timestamp")
        plt.ylabel("Value")
        plt.grid(True)
        plt.draw()
        plt.savefig('C:/Users/g.souza/PycharmProjects/EPMProcessorTestes/Graphs/'+str(name)+'.png')
        
        maximo = value.max()
        minimo = value.min()
        media = value.mean()
        mediamovel = y_av.mean()
        totalpontos = len(value)
        primeiradata = timestamp[0]
        ultimadata = timestamp[-1]
        tempomedicao = (ultimadata - primeiradata)
        tempototal = "%d dias e %d horas"%(tempomedicao.days, (round(tempomedicao.seconds/3600)))
        fim = time.time()
        texec = (fim - ini)
        
        with open('C:/Users/g.souza/PycharmProjects/EPMProcessorTestes/InfosVariaveis.py', 'a') as arq:
            novosvalores = "caracteristicas[" + "\"" + name + "\"" + "]={'max':"+" "+str(maximo)+", 'min':"+" "+str(minimo)+", 'media':"+" "+str(media)+", 'mediamovel':"+" "+str(mediamovel)+", 'tamanho':"+" "+str(totalpontos)+", 't(s)exec':"+" "+str(texec)+", 'primeiroponto':" + " \'" + str(primeiradata) + "\'" + ", 'ultimoponto':" + " \'" + str(ultimadata) + "\'" + ", 'tempo de medicao':" + " \'" + str(tempototal) + "\'" + "}\n"
            arq.write(novosvalores)
conn.close()